# Project T Final: PCA and CCA

By Jai Bansal, Abhinav Gopal, Grace Kull, William McEachen, Shrey Vasavada

In [ ]:
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.cross_decomposition import CCA
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.preprocessing import StandardScaler

Run the cell below only once per session. The utils.py file contains some black box functions which will be used later in this assignment. 

In [ ]:
!ln -s ./utils.py utils.py

In [ ]:
import utils

## Part 1: Initial Data Cleaning

In this assignment, we will be looking at a real dataset of baseball players, using their game statistics to predict their salaries. Some of the information in this dataset about the players includes their names, ids, positions, years played, and their batting statistics. 

First, load the csv file to see all the information.

In [ ]:
baseball = pd.read_csv("baseball_data.csv")

In [ ]:
baseball.head()

From the initial obsevation of the data, it is clear that the types of data needed to predict salaries will be numerical or categorical. Additionally, rows with no salary information will have no use to us. So we'll start the data cleaning process by dropping columns that don't have categorical or numerical data, and dropping rows with no salary information. The columns being dropped are 'id', 'teamid', 'yearid', 'lgid','namefirst','namelast', and 'g_batting'.

In [ ]:
data = baseball.drop(columns = ['id', 'teamid', 'yearid','lgid','namefirst','namelast', 'g_batting'])
data = data[data['salary'] > 0]

We will now start one-hot encoding our position data. Because there are several different positions, where each one can have an impact on a player's salary, it is important for us to take that data into account, even though it's not numerical. We will create a new column for each different kind of position and for each row, the value of that column will be 1 if the player is in that position, and 0 if the player is not. Fill out the cell below to implement one-hot encoding. Remember to drop the original 'pos' column from the dataset when the one-hot encoding is complete.

In [ ]:
# One-hot encoding for each of the different positions

# YOUR CODE HERE #

data_one_hot = data.drop(columns = ['pos'])


Finally, replace all NaN values with their column mean.

In [ ]:
# YOUR CODE HERE #

In [ ]:
# Cleaned data from one-hot encoding the positions. 
cleaned.head(10)

## Part 2: Linear Regression, SVD, and PCA

Now that the data is cleaned, we can explore different techniques that can be used to predict the different salaries. We will also examine how well each technique performs in salary prediction, and intuition behind each technique.

First, we split the data into the set of features and the value that is being predicted, which are the salaries. 

In [ ]:
X, y = cleaned.drop(columns=['salary'], axis=1), cleaned['salary']

### 2a. Linear Regression

Linear regression is one of the easiest methods to make a predictive model. \
It is a linear method that models the relationship between one or more features (independent variables)
and an outcome variable (dependent variable). Least squares is a common method within linear regression 
that finds the feature weights that will lead to the best-fit linear regression model. The formula for
Least Squares is $y = \mathbf{X}w$, where $y$ is our outcome, $\mathbf{X}$ is our set of features, and 
$w$ is the weight for each feature. The weights can be calculated using the formula 
$\textbf{w} = (\mathbf{X}^{T}\mathbf{X})^{-1}\mathbf{X}^{T}y$. 

Fill out the cell below to implement the least squares function that returns the weights given by the above formula. You may not use sklearn or NumPy's built-in least squares function for this.

In [ ]:
def least_squares(features, prediction):
    # YOUR CODE HERE #

Now run your <code>least_squares</code> function on the data above, and calculate the MSE for the data. Hint: use <code>np.matrix</code>

In [ ]:
weights = least_squares(X, y)
mse_linear_reg = # YOUR CODE HERE #
mse_linear_reg

We will now explore some methods to see how they affect the MSE, starting with PCA below.

### 2b. Singular Value Decomposition (SVD)

Singular Value Decomposition (SVD) is a method used to decompose an $m x n$ matrix $A$ into the form $UDV^T$, where $U$ is $mxm$, $D$ is $mxn$ and $V^T$ is $nxn$. It is used in several different real-world applications including low-rank approximation and image compression.

In order to find the SVD of $A$, find the eigenvalue/vector pairs of $A^TA$, and then take the square roots of the eigenvalues to get the singular values of A. Finally, sort the singular values in descending order, and have the corresponding eigenvectors match that sorted order. Then find the eigenvectors of $AA^T$ using the singular values. Then normalize the eigenvectors of $A^TA$ to get the $V$ matrix, insert the sorted singular values into a zero-padded diagonal matrix $D$, and then normalize the eigenvectors of $AA^T$ to get the $U$ matrix.

First, fill out the following cell to plot the singular values of the cleaned dataset. Hint: <code>np.linalg.svd</code> might be useful

In [ ]:
singular_values = # YOUR CODE HERE #
x_labels = # YOUR CODE HERE #
# The y axis is on a log scale to better see the difference in sizes between the eigenvalues
plt.figure(figsize=[10, 6])
plt.yscale("log")
plt.scatter(x_labels, singular_values)
plt.title("Singular values of the cleaned dataset")
plt.xlabel("Singular value index")
plt.ylabel("Magnitude")

Now fill out the following cell to implement <code>svd_manual</code>. You may not use <code>np.linalg.svd</code>

In [ ]:
# Helper function that returns the matrix with singular values on the diagonal and zeros elsewhere
def get_diag(rows, cols, values):
    mat = np.zeros((rows, cols))
    index = 0
    for i in values:
        mat[index][index] = i
        index += 1
    return mat

# Computes the SVD of matrix X
def svd_manual(X):
    # YOUR CODE HERE #
    
    
    # If the number of rows is greater than the number of columns, transpose the matrices before returning #
    if (len(X) > len(X[0])):
        U, D, V = V.T, D.T, U.T
    return U, D, V

u, d, v_t = svd_manual(np.array(X))

Now we will see what happens when only the first $k$ singular values are used to approximate the original matrix. As seen in the plot above, the magnitude of the singualr levels off after only a few of them. As a result, the approximation of the original matrix of only a few singular values may not be far off from an approximation using most of the singular values. In fact, when approximating a matrix A with a rank-k matrix $A_k$, the Eckart–Young–Mirsky theorem states that for a matrix $A = UDV^T$ with rank r, for any integer k, where $0\leq k\leq r$, the best rank-k approximation of $A$ is $A_k = \sum_{i=1}^{k} \sigma_i u_i v_i^T$, where $u_i$ and $v_i$ denote the $i$th columns of $\textbf{U}$ and $\textbf{V}$, respectively. 

Fill out the cell below to implement <code>svd_k_approx</code>. You may use <code>np.linalg.svd</code> in your solution.

In [ ]:
# Returns the best rank-k approximation of X
def svd_k_approx(X, k):
    # if the rank of X is less than or equal to k, return X
    if np.linalg.matrix_rank(X) <= k:
        return X
    else:
        # YOUR CODE HERE #

While SVD can make use of the singular values with the greatest influence, in the next part, we will look at how PCA makes use of the directions with the greatest influence.

### 2c. Principal Component Analysis (PCA)

Principal Component Analysis (PCA) is a method that makes use of the "orthogonal directions" that have the most influence on a dataset. It is useful because often only a few components are needed in order to make accurate predictions while components with less weight are barely influential.

To implement PCA, it it important to first scale the features, in order for certain features to not overpower other features. Next, the mean vector is calculated using the formula $\textbf{m} =\frac{1}{n}\sum_{i=1}^{n} x_i$. Next, calculate the scatter matrix, which is $S = \sum_{i=1}^{n} (x_i - \textbf{m})(x_i - \textbf{m})^T$. Then find the eigenvalues and vectors of the scatter matrix, take the first num_components of the eigenvectors, and calculate the projection matrix using those eigenvectors and the scaled matrix. Finally, perform linear regression with the projection matrix.

Fill out the cell below to calculate the MSE for linear regresion with `num_components` features. You may only use <code>numpy</code> and not <code>PCA</code> or <code>LinearRegression</code> in this part of your solution.

In [ ]:
def pca(features, preds, num_components):
    # First, the features are scaled since no feature is more important 
    scaled = StandardScaler().fit_transform(features)
    mean_vector = np.zeros(51)
    # YOUR CODE HERE #
    scatter_matrix = np.zeros((51, 51))
    # YOUR CODE HERE #
    eigenvalues, eigenvectors = np.linalg.eig(scatter_matrix)
    # This line is included to account for slight complex values being included in the scatter matrix calculations
    eigenvectors = eigenvectors.real
    # YOUR CODE HERE #
    return mse

Fill out the following cells below to find the MSE using PCA with linear regression for 2, 5, and 8 components.

In [ ]:
pca(X, y, 2)

In [ ]:
pca(X, y, 5)

In [ ]:
pca(X, y, 8)

What number of components had the smallest MSE? Does increasing the number of components lower the MSE, or does it increase after a certain amount of components? Will PCA ever produce a MSE lower that that of using linear regression alone?

In [ ]:
# YOUR OBSERVATIONS HERE #

Now use the <code>PCA</code> and `LinearRegression()` modules from <code>sklearn</code> to check your implementation of PCA. First, we'll write a function to calculate MSE from an OLS model with `num_components` number of features.

In [ ]:
def sklearn_PCA(features, preds, num_components):
    # YOUR CODE HERE #

In the following 3 cells, run this function with 2, 5, and 8 features.

In [ ]:
sklearn_PCA(X, y, 2)

In [ ]:
sklearn_PCA(X, y, 5)

In [ ]:
sklearn_PCA(X, y, 8)

Now, graph the number of components, versus the MSE, for PCA.

In [ ]:
X_plot = [i for i in range(1, X.shape[1])]
Y_plot = []
for i in X_plot:
    pca_i = PCA(n_components=i)
    pca_i.fit(X, y)
    pca_i_X = pca_i.transform(X)
    reg_i = LinearRegression().fit(pca_i_X, y)
    Y_plot.append(np.mean((reg_i.predict(pca_i_X) - y)**2) ** 0.5)
plt.plot(X_plot, Y_plot)
plt.xlabel("Number of components")
plt.ylabel("MSE")
plt.title("Number of PCA components vs. MSE")

Fill out the cell below with your observations:

In [ ]:
# YOUR OBSERVATIONS HERE #

## Part 3: Noisy Data

In the previous part, we saw observations from Linear Regression, PCA, and CCA on a dataset that has not been affected by noise. We will now observe what happens when our salary column has noise added to it, and how the accuracy of our predictions are affected by it.

First, run the cell below to generate noised predictions. The <code>utils.noised_predictions</code> is a black box function that will randomly add noise to our salary column using a random Gaussian variable, which is a random variable with mean $\mu$ and standard deviation $\sigma$. It has the probability density function $f(x) = \frac{1}{\sigma \sqrt{2\pi}}e^{\frac{-1}{2}(\frac{(x - \mu)}{\sigma})^2}$

In [ ]:
y_noise = utils.noised_predictions(cleaned)
y_noise

### 3a. Noisy PCA

Use `sklearn_PCA` with your noisy predictions and 2, 5, and 8 of the features from the original dataset. How does adding noise to the salary column change the MSE?

In [ ]:
# PCA with 2 components #
# YOUR CODE HERE #

In [ ]:
# PCA with 5 components #
# YOUR CODE HERE #

In [ ]:
# PCA with 8 components #
# YOUR CODE HERE #

Now, graph the number of components, versus the MSE, for the noisy PCA.

In [ ]:
X_plot = [i for i in range(1, X.shape[1])]
Y_plot = []
for i in X_plot:
    pca_i = PCA(n_components=i)
    pca_i.fit(X, y_noise)
    pca_i_X = pca_i.transform(X)
    reg_i = LinearRegression().fit(pca_i_X, y_noise)
    Y_plot.append(np.mean((reg_i.predict(pca_i_X) - y_noise)**2) ** 0.5)
plt.plot(X_plot, Y_plot)
plt.xlabel("Number of components")
plt.ylabel("MSE")
plt.title("Number of PCA components vs. Noisy MSE")

Fill out the cell below with your observations. How does adding noise to the salary column change the MSE? How does this plot compare to that of the PCA plot without noise?

In [ ]:
# YOUR OBSERVATIONS HERE #

## Part 4: Dirty Data and CCA
In industry contexts, you will often rely on remote data that has been collected in some black box form. This section will work through the process of uncovering this data and cleaning it. The following call creates a `hw.db` file in the same directory as this notebook. 

In [ ]:
utils.make_db('baseball_data.csv')

### 4a. Cleaning the dirty data

We now have a SQLite DB we can access. Connect to this database and create a pandas DataFrame from the table within it.

In [ ]:
import sqlite3
# YOUR CODE HERE #

Let's explore this data:

In [ ]:
dirty_data.head()

As you can see, this data has several issues: there are nan values, duplicate rows, column name changes, and data format changes. Let's clean this data!

First, let's standardize the column labels. (Hint: how can we create a predictable format invariant to the headers' current form?)

In [ ]:
# YOUR CODE HERE #

Next, drop duplicates. (Hint: some columns have no Nan values).

In [ ]:
# YOUR CODE HERE #

Replace NaN values with their column mean. This requires setting columns with string values back to integers.

In [ ]:
# YOUR CODE HERE #

To prepare our data, one-hot-encode categorical variables.

In [ ]:
ohe_dirty_data = dirty_data.copy()
for col in ('POS', 'TEAMID', 'LGID'):
    ohe_dirty_data = ohe_dirty_data.merge(
    pd.get_dummies(dirty_data[col]),
     how='inner', right_index=True, left_index = True
    ).T.drop_duplicates().T.rename(lambda c: str(c).rsplit('_x', 1)[0], axis='columns').drop(col, axis=1)



We don't need player names/ID, so remove those features (they aren't useful categorical features). Additionally, drop the 'INDEX' column because it is a residual from importing SQL data. Because `G_BATTING` is all nan, r

In [ ]:
# YOUR CODE HERE #

In [ ]:
ohe_dirty_data.isna().sum()[ohe_dirty_data.isna().sum().astype(bool)]

With our data clean, let's split our data into our features and prediction column.

In [ ]:
X_dirty = ohe_dirty_data.drop(columns=['SALARY'])
y_dirty = ohe_dirty_data['SALARY']

### 4b. Dirty PCA

Now run PCA on your dirty dataset, using 2, 5, and 8 components and find the MSE for your predictions. How does the performance compare to PCA of the original cleaned dataset?

In [ ]:
# PCA with 2 components #
# YOUR CODE HERE #

In [ ]:
# PCA with 5 components #
# YOUR CODE HERE #

In [ ]:
# PCA with 8 components #
# YOUR CODE HERE #

Now, graph the number of components, versus the MSE, for the dirty PCA.

In [ ]:
X_plot = [i for i in range(1, X_dirty.shape[1])]
Y_plot = []
for i in X_plot:
    pca_i = PCA(n_components=i)
    pca_i.fit(X_dirty, y_dirty)
    pca_i_X = pca_i.transform(X_dirty)
    reg_i = LinearRegression().fit(pca_i_X, y_dirty)
    Y_plot.append(np.mean((reg_i.predict(pca_i_X) - y_dirty)**2) ** 0.5)
plt.plot(X_plot, Y_plot)
plt.xlabel("Number of components")
plt.ylabel("MSE")
plt.title("Number of Dirty PCA components vs. Dirty MSE")

Fill out the cell below with your observations. How does the MSE change for the dirty dataset? How does the overall plot shape change?

In [ ]:
# YOUR OBSERVATIONS HERE #

### 4c. Canonical Correlation Analysis (CCA) and Dirty Data

From part 2, we saw that PCA is an unsupervised algorithm because it does not rely on feature labels, but rather focuses on the directions with the greatest variance in the feature matrix. However, there are some situations where the most relevant dimensions are not those with the greatest variance. For example, if the feature data was contaminated with a strong, correlated noise signal, PCA would actually throw away those dimensions with this strong noise variation, the opposite of our desired outcome. In this case, we prefer to approach dimensionality reduction in a way that takes advantage of paired, i.e labeled $(x, y)$ data. This is where CCA comes into play.

For two vector quantities with many paired samples, $X$ and $Y$, that are jointly distributed as random variables, our goal is to understand the relationship between $X$ and $Y$. In this context however, we would like to understand the relationship between the dirty dataset and the original dataset, and how it affects the MSE when predicting salaries.

Fill out the cells below to implement <code>sklearn_CCA</code>, which will make use of <code>CCA</code> along with <code>LinearRegression</code> to return the MSE.

In [ ]:
def sklearn_CCA(X, Y, preds, num_components):
    # YOUR CODE HERE #

Now run <code>sklearn_CCA</code> with 2, 5, and 8 components. Your Y argument will be the <code>dirty_421</code> variable below, which is randomly sampled rows from the dirty dataset. This is needed because X and Y must have the same number of samples.

In [ ]:
dirty_421 = X_dirty.sample(n=421)

In [ ]:
# CCA with 2 components #
# YOUR CODE HERE #

In [ ]:
# CCA with 5 components #
# YOUR CODE HERE #

In [ ]:
# CCA with 8 components #
# YOUR CODE HERE #

Now run the following cell to plot the MSE vs. the number of CCA components. 

In [ ]:
X_plot = [i for i in range(1, X.shape[1])]
Y_plot = []
for i in X_plot:
    cca_i = CCA(n_components=i)
    cca_i.fit(X, dirty_421)
    cca_i_X = cca_i.transform(X)
    reg_i = LinearRegression().fit(cca_i_X, y)
    Y_plot.append(np.mean((reg_i.predict(cca_i_X) - y)**2) ** 0.5)
plt.plot(X_plot, Y_plot)
plt.xlabel("Number of components")
plt.ylabel("MSE")
plt.title("Number of Dirty CCA components vs. Dirty MSE")

Fill out the cell below with your observations. How does CCA perform compared to PCA with the same number of components? How does increasing the number of components affect the MSE?

In [ ]:
# YOUR OBSERVATIONS HERE #